## Import des données

In [12]:
from skimage import io

In [6]:
import os

folder_directory = "../data/ChicagoFacesData"
list_files = []

for path, subdirs, files in os.walk(folder_directory):
    for name in files:
        if name[-4:]=='.jpg':
            #print(os.path.join(path, name))
            list_files.append(os.path.join(path, name))
print(list_files)

['../data/ChicagoFacesData\\AF-200\\CFD-AF-200-228-N-thumbnail.jpg', '../data/ChicagoFacesData\\AF-200\\CFD-AF-200-228-N.jpg', '../data/ChicagoFacesData\\AF-201\\CFD-AF-201-060-N.jpg', '../data/ChicagoFacesData\\AF-202\\CFD-AF-202-122-N.jpg', '../data/ChicagoFacesData\\AF-203\\CFD-AF-203-077-N.jpg', '../data/ChicagoFacesData\\AF-204\\CFD-AF-204-067-N.jpg', '../data/ChicagoFacesData\\AF-205\\CFD-AF-205-155-N.jpg', '../data/ChicagoFacesData\\AF-206\\CFD-AF-206-079-N.jpg', '../data/ChicagoFacesData\\AF-207\\CFD-AF-207-023-N.jpg', '../data/ChicagoFacesData\\AF-208\\CFD-AF-208-003-N.jpg', '../data/ChicagoFacesData\\AF-209\\CFD-AF-209-006-N.jpg', '../data/ChicagoFacesData\\AF-210\\CFD-AF-210-050-N.jpg', '../data/ChicagoFacesData\\AF-211\\CFD-AF-211-066-N.jpg', '../data/ChicagoFacesData\\AF-212\\CFD-AF-212-097-N.jpg', '../data/ChicagoFacesData\\AF-213\\CFD-AF-213-126-N.jpg', '../data/ChicagoFacesData\\AF-214\\CFD-AF-214-139-N.jpg', '../data/ChicagoFacesData\\AF-215\\CFD-AF-215-70-N.jpg', '../

In [7]:
from PIL import Image

In [8]:
im = Image.open(list_files[1])
im.show()


In [9]:
class ChicagoFacesDataset:
    """Chicago Faces dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.landmarks_frame = pd.read_csv(csv_file)
        #self.informations_csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        self.init_get_path_files()

    def init_get_path_files(self):
        self.list_files = []
        folder_directory = "../data/ChicagoFacesData"

        for path, subdirs, files in os.walk(folder_directory):
            for name in files:
                if name[-4:] == '.jpg':
                    # print(os.path.join(path, name))
                    self.list_files.append(os.path.join(path, name))
        print(self.list_files)

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, idx):
        #img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx, 0])
        img_name = os.path.join(self.list_files[idx])
        image = io.imread(img_name)
        #landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        #ample = {'image': image, 'landmarks': landmarks}
        sample = {'image': image}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [10]:
faces_dataset = ChicagoFacesDataset(csv_file='../data/ChicagoFacesData/CFD_2.0.3_Norming_Data_and_Codebook.csv',
                                    root_dir='../data/ChicagoFacesData/')



['../data/ChicagoFacesData\\AF-200\\CFD-AF-200-228-N-thumbnail.jpg', '../data/ChicagoFacesData\\AF-200\\CFD-AF-200-228-N.jpg', '../data/ChicagoFacesData\\AF-201\\CFD-AF-201-060-N.jpg', '../data/ChicagoFacesData\\AF-202\\CFD-AF-202-122-N.jpg', '../data/ChicagoFacesData\\AF-203\\CFD-AF-203-077-N.jpg', '../data/ChicagoFacesData\\AF-204\\CFD-AF-204-067-N.jpg', '../data/ChicagoFacesData\\AF-205\\CFD-AF-205-155-N.jpg', '../data/ChicagoFacesData\\AF-206\\CFD-AF-206-079-N.jpg', '../data/ChicagoFacesData\\AF-207\\CFD-AF-207-023-N.jpg', '../data/ChicagoFacesData\\AF-208\\CFD-AF-208-003-N.jpg', '../data/ChicagoFacesData\\AF-209\\CFD-AF-209-006-N.jpg', '../data/ChicagoFacesData\\AF-210\\CFD-AF-210-050-N.jpg', '../data/ChicagoFacesData\\AF-211\\CFD-AF-211-066-N.jpg', '../data/ChicagoFacesData\\AF-212\\CFD-AF-212-097-N.jpg', '../data/ChicagoFacesData\\AF-213\\CFD-AF-213-126-N.jpg', '../data/ChicagoFacesData\\AF-214\\CFD-AF-214-139-N.jpg', '../data/ChicagoFacesData\\AF-215\\CFD-AF-215-70-N.jpg', '../

In [11]:
sample = faces_dataset[4]
sample.show()

NameError: name 'io' is not defined